In [1]:
from lmfit import Parameters
import matplotlib
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.signal import wiener, filtfilt, butter, gaussian, freqz
from scipy.ndimage import filters
from core.util.units import compute_dft

In [4]:
def poissonian_testing():
    start=0
    stop=30
    mu=8
    num_points=1000
    x = np.array(np.linspace(start, stop, num_points))
#            x = np.array(x,dtype=np.int64)
    mod, params = fitlogic.make_poissonian_model()
    print('Parameters of the model',mod.param_names)

    p=Parameters()
    p.add('mu',value=mu)
    p.add('amplitude',value=200.)

    data_noisy=(mod.eval(x=x,params=p) *
                np.array((1+0.001*np.random.normal(size=x.shape) *
                p['amplitude'].value ) ) )

    print('all int',all(isinstance(item, (np.int32,int, np.int64)) for item in x))
    print('int',isinstance(x[1], int),float(x[1]).is_integer())
    print(type(x[1]))
    #make the filter an extra function shared and usable for other functions
    gaus=gaussian(10,10)
    data_smooth = filters.convolve1d(data_noisy, gaus/gaus.sum(),mode='mirror')


    result = fitlogic.make_poissonian_fit(x, data_noisy, estimator=fitlogic.estimate_poissonian)
    print(result.fit_report())

    plt.figure()
    plt.plot(x, data_noisy, '-b', label='noisy data')
    plt.plot(x, data_smooth, '-g', label='smoothed data')
    plt.plot(x,result.init_fit,'-y', label='initial values')
    plt.plot(x,result.best_fit,'-r',linewidth=2.0, label='fit')
    plt.xlabel('counts')
    plt.ylabel('occurences')
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
    plt.show()

In [5]:
poissonian_testing()

Parameters of the model ['amplitude', 'mu']
all int False
int False False
<class 'numpy.float64'>
[[Model]]
    (Model(amplitude_function) * Model(poisson_function))
[[Fit Statistics]]
    # function evals   = 15
    # data points      = 1000
    # variables        = 2
    chi-square         = 5877.517
    reduced chi-square = 5.889
[[Variables]]
    amplitude:   198.975937 +/- 1.335744 (0.67%) (init= 217.6834)
    mu:          7.97120363 +/- 0.026581 (0.33%) (init= 7.447447)
[[Correlations]] (unreported correlations are <  0.100)
    C(amplitude, mu)             =  0.126 



In [12]:
def double_poissonian_testing():
    """ Testing of double poissonian with self created data.
    First version of double poissonian fit."""

    start=100
    stop=300
    num_points=int((stop-start)+1)*100
    x = np.linspace(start, stop, num_points)

    # double poissonian
    mod,params = fitlogic.make_poissoniandouble_model()
    print('Parameters of the model',mod.param_names)
    parameter=Parameters()
    parameter.add('p0_mu',value=200)
    parameter.add('p1_mu',value=240)
    parameter.add('p0_amplitude',value=1)
    parameter.add('p1_amplitude',value=1)
    data_noisy = ( np.array(mod.eval(x=x,params=parameter)) *
                   np.array((1+0.2*np.random.normal(size=x.shape) )*
                   parameter['p1_amplitude'].value) )


    #make the filter an extra function shared and usable for other functions
    gaus=gaussian(10,10)
    data_smooth = filters.convolve1d(data_noisy, gaus/gaus.sum(),mode='mirror')

    result = fitlogic.make_poissoniandouble_fit(x, data_noisy, estimator=fitlogic.estimate_poissoniandouble)
    print(result.fit_report())

    plt.figure()
    plt.plot(x, data_noisy, '-b', label='noisy data')
    plt.plot(x, data_smooth, '-g', label='smoothed data')
    plt.plot(x,result.init_fit,'-y', label='initial values')
    plt.plot(x,result.best_fit,'-r',linewidth=2.0, label='fit')
    plt.xlabel('counts')
    plt.ylabel('occurences')
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0.)
    plt.show()

In [13]:
double_poissonian_testing()

Traceback (most recent call last):
  File "/home/jan/qo/qudi/logic/jupyterkernel/qzmqkernel.py", line 795, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-4c7ef16d294d>", line 1, in <module>
    double_poissonian_testing()
  File "<ipython-input-12-4367317c7e35>", line 11, in double_poissonian_testing
    mod,params = fitlogic.make_poissoniandouble_model()
  File "/home/jan/qo/qudi/logic/fitmethods/poissonianlikemethods.py", line 148, in make_poissoniandouble_model
    return self.make_multiplepoissonian_model(2)
  File "/home/jan/qo/qudi/core/base.py", line 138, in __getattr__
    return Fysom.__getattr__(self, name)
  File "/home/jan/qo/qudi/core/FysomAdapter.py", line 36, in __getattr__
    self.__class__, name))
AttributeError: '<class 'logic.fit_logic.FitLogic'>' object has no attribute 'make_multiplepoissonian_model'
